In [17]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [45]:
npi_data = pd.read_csv('data/npidata_pfile_20050523-20220213.csv', 
                          nrows = 100)

In [47]:
npi_data.to_csv("npi_test_view.csv", sep='\t')

In [11]:
npi_data.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1679576722,1.0,NaN,NaN,NaN,WIEBE,DAVID,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,NaN,NaN,PILCHER,WILLIAM,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,NaN,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
hop_team = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', 
                          nrows = 100)

In [8]:
hop_team.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880


In [ ]:
#Just Taxonomy Group Columns
#'Healthcare Provider Taxonomy Group_1', 'Healthcare Provider Taxonomy Group_2', 'Healthcare Provider Taxonomy Group_3', 'Healthcare Provider Taxonomy Group_4', 'Healthcare Provider Taxonomy Group_5', 'Healthcare Provider Taxonomy Group_6','Healthcare Provider Taxonomy Group_7', 'Healthcare Provider Taxonomy Group_8', 'Healthcare Provider Taxonomy Group_9','Healthcare Provider Taxonomy Group_10', 'Healthcare Provider Taxonomy Group_11', 'Healthcare Provider Taxonomy Group_12', 'Healthcare Provider Taxonomy Group_13', 'Healthcare Provider Taxonomy Group_14', 'Healthcare Provider Taxonomy Group_15']]

In [ ]:
#Columns for Taxonomy Codes and Switches
#'Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10' ,'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15'

In [76]:
chunks = pd.read_csv('data/npidata_pfile_20050523-20220213.csv', chunksize = 10000)
switch = ['Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Primary Taxonomy Switch_15']
code = ['Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Taxonomy Code_6','Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Taxonomy Code_9','Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Taxonomy Code_15']
sf_list = []
x = 1
for chunk in chunks:
    if (x < 5):
        sf = chunk[['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)','Provider Last Name (Legal Name)','Provider First Name','Provider Middle Name','Provider Name Prefix Text','Provider Name Suffix Text','Provider Credential Text','Provider First Line Business Practice Location Address','Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code','Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10' ,'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15']]
        for s, c in zip(switch, code):
            sf.loc[sf[s]=='Y','Combined Taxonomy'] = sf[c]
        sf['Combined Taxonomy'].fillna('Other',inplace=True)
        sf_trim = sf[['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)','Provider Last Name (Legal Name)','Provider First Name','Provider Middle Name','Provider Name Prefix Text','Provider Name Suffix Text','Provider Credential Text','Provider First Line Business Practice Location Address','Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code','Combined Taxonomy']]
        sf_list.append(sf_trim)
        shots_fired = pd.concat(sf_list)
        x = x + 1
    else:
        break

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17,76,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,155,163,171,175,178,179,183,186,187,189,190,191,193,194,195,197,198,199,201,202,205,206,209,211,213,218,221,223,225,227,229,233,237,241,245,247,249,253,257,261,265,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17,76,84,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,167,171,175,179,183,187,191,193,194,195,197,198,201,202,205,206,209,210,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\cmerr\anaconda3\lib\site-package

In [78]:
db = sqlite3.connect('data/test_npi3_chunk.sqlite')

switch = ['Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Primary Taxonomy Switch_15']
code = ['Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Taxonomy Code_6','Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Taxonomy Code_9','Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Taxonomy Code_15']

for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20220213.csv', chunksize = 5000)):
    sf = chunk[['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)','Provider Last Name (Legal Name)','Provider First Name','Provider Middle Name','Provider Name Prefix Text','Provider Name Suffix Text','Provider Credential Text','Provider First Line Business Practice Location Address','Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code','Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10' ,'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15']]
    
    #loops through switch numbers and code numbers simultaneously and simplifies the data down to one column
    for s, c in zip(switch, code):
        sf.loc[sf[s]=='Y','Combined Taxonomy'] = sf[c]
        
    sf['Combined Taxonomy'].fillna('Other',inplace=True)
    #removes the 15 taxonomy code columns and the 15 taxonomy switch columns
    sf_trim = sf[['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)','Provider Last Name (Legal Name)','Provider First Name','Provider Middle Name','Provider Name Prefix Text','Provider Name Suffix Text','Provider Credential Text','Provider First Line Business Practice Location Address','Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code','Combined Taxonomy']]
    
    sf_trim.to_sql('npi_data', db, if_exists = 'append', index = False) 

0it [00:00, ?it/s]

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,34,76,80,84,85,89,93,101,105,155,163,167,170,171,175,177,179,181,182,183,185,186,189,190,193,194,197,198,201,205,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\cmerr\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,34,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,93,94,95,96,97,98,99,100,101,102,103,105,106,159,167,169,171,179,183,185,187,191,195,199,201,202,205,206,209,210,213,214,215,217,218,221,225,226,227,230,321,322,323,324,325) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,32,80,84,85,88,89,91,93,94,95,98,99,102,161,169,171,177,181,185,189,193,197,323,324,325,326,327) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (24,32,80,81,84,85,88,89,91,92,93,94,95,96,97,98,99,100,101,102,103,104,106,159,167,169,173,174,177

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (24,32,35,68,69,72,73,76,77,79,80,81,82,83,85,86,87,88,89,90,91,92,93,94,95,98,99,102,103,105,106,163,167,173,174,177,178,181,182,185,186,187,189,191,193,194,195,197,198,201,202,203,205,206,209,210,211,213,214,215,217,218,219,221,222,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,34,46,68,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,89,90,91,94,95,98,99,102,103,106,163,167,169,171,173,174,175,177,178,181,182,185,186,187,189,190,320,321,322,323,324,325) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: Dtyp

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,27,32,35,40,72,76,80,84,87,88,89,90,91,92,93,94,95,97,98,99,101,102,103,106,159,167,171,174,175,178,181,185,186,189,190,193,194,195,197,199,201,203,205,207,209,211,213,217,218,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (24,34,35,76,80,81,84,85,88,89,91,92,93,94,95,96,97,98,99,100,101,102,103,105,106,159,163,167,175,177,178,312,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (27,32,35,40,76,80,81,84,85,89,93,97,143,159,175,177,178,181,182,183,185,186,189,190,193,194,19

...

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (24,40,72,73,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,98,99,102,103,106,115,117,121,122,125,126,127,129,130,133,134,319,320,321,322,323,324) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,40,60,65,69,72,73,77,80,81,84,85,88,89,95,98,99,102,103,106,117,118,119,121,122,125,126,129,320,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,60,64,65,67,69,70,71,73,74,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106,115,117,118,119,121,122,312,319,320,321,322,323,324,325,326,

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (24,40,60,61,75,78,79,82,83,86,115,117,118,121,122,125,126,129,130,319,320,321,322) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,60,64,65,69,75,78,79,82,83,86,87,90,91,94,111,115,121,122,125,129,321,322,323) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,39,40,60,64,68,69,83,86,87,90,91,94,95,98,114,115,117,118,119,121,123,125,319,321,322) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,40,60,64,68,69,72,73,77,83,86,87,89,90,91,93,94,95,97,98,111,115,117,118,119,121,122,125,127,129,133,135,137,321,322,323,324,325,326) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,64,65,87,90,91,94,95,98,99,102,103,106,115,118,119,122,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (24,32,40,64,68,69,72,73,77,99,102,115,117,118,119,121,122,312,321,322,325) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,40,60,61,64,65,69,73,75,76,77,78,79,80,81,82,83,86,87,90,91,94,95,98,99,102,118,119,121,122,123,125,126,129,130,131,133,134,137,317,318,319,320,321,322,323,324,325,326,327) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (24,32,39,40,60,65,67,70,71,74,75,78,79,82,83,86,87,90,91,94,95,98,99,102,115,119,121,122,318,319,320,321,322,323,324,325,326,327) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,40,60,64,68,69,73,83,86,87,90,91,94,95,98,99,102,103,106,115,118,121,122,323,324,325,326,327) have mixed types.Specify dtyp

C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,39,60,61,79,82,83,86,87,90,91,94,95,98,99,102,103,106,111,114,115,117,118,121,312,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,40,64,68,73,76,77,80,81,85,87,90,91,94,95,98,99,102,103,106,111,115,121,122,125,319,320,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\cmerr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,24,32,60,64,68,69,73,77,81,85,87,89,90,91,94,95,98,99,102,103,106,111,115,118,121,122,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  exec

In [53]:
db = sqlite3.connect('data/test_npi3_chunk.sqlite')

query = """

        """

shots_sqlite = pd.read_sql(query, db)

db.close()